<a href="https://colab.research.google.com/github/andy258741/NCHU_Finance/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 期末專題流程
### 1. 蒐集資料
##### 1_1. 蒐集半導體產業個股的每日交易資訊
##### 1_2. 蒐集半導體產業個股相關新聞
### 2. 特徵生成
##### 2_1. 生成個股基礎風險指數
##### 2_2. 生成個股之回報率
##### 2_3. 生成個股風險分數
### 3. 投資組合選擇
##### 3_1. 基因演算法
### 4.實驗與結果分析

### 1. 蒐集資料
##### 爬取半導體業個股之資料

In [ ]:
# 安裝所需套件
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install OpenAI

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Fetched 4,225 kB in 4s (1,136 kB/s)
Reading pack

In [ ]:
# 透過yahoo奇摩股市爬蟲得知半導體產業的所有上市公司代碼
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def get_all_stocks(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 模擬下拉，否則只會讀取到30家公司的代號
        time.sleep(1.5)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    stocks = []
    find_div = soup.find_all('li', {'class': 'List(n)'})
    for i in find_div:
        num = i.find('div', {'class': 'D(f) Ai(c)'})
        if num and num.text:
            stocks.append(num.text.split(".")[0])

    driver.quit()
    return stocks

url = "https://tw.stock.yahoo.com/class-quote?sectorId=40&exchange=TAI" # 上市

stocks_listed = get_all_stocks(url)
print("上市公司數量:", len(stocks_listed), "上市:", stocks_listed)

上市公司數量: 84 上市: ['2302', '2303', '2329', '2330', '2337', '2338', '2340', '2342', '2344', '2351', '2363', '2369', '2379', '2388', '2401', '2408', '2434', '2436', '2441', '2449', '2451', '2454', '2458', '2481', '3006', '3014', '3016', '3034', '3035', '3041', '3094', '3189', '3257', '3413', '3443', '3450', '3530', '3532', '3545', '3583', '3588', '3592', '3661', '3686', '3711', '4919', '4952', '4961', '4967', '4968', '5222', '5269', '5285', '5471', '6202', '6239', '6243', '6257', '6271', '6415', '6451', '6515', '6525', '6526', '6531', '6533', '6552', '6573', '6695', '6719', '6756', '6770', '6789', '6799', '6937', '6962', '8016', '8028', '8081', '8110', '8131', '8150', '8261', '8271']


In [ ]:
# 建立一個字典，以儲存所有資訊
stock_dic = {i: {"類別":"上市"} for i in stocks_listed}

In [ ]:
# 1_1.使用台灣證交所的api得到個股的交易資訊
import pandas as pd
import requests
from datetime import datetime

# 將民國年轉換為西元年
def convert_date(date_str):
    year, month, day = date_str.split('/')
    year = str(int(year) + 1911)  # 民國年轉西元年
    return f"{year}/{month}/{day}"

def fetch_monthly_data(stock_no, year, month):
    # 定義日期，使用每月的第一天作為請求參數
    date_str = f"{year}{month:02d}01"
    url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}"

    # 發送 API 請求
    response = requests.get(url)

    # 如果請求成功，返回JSON數據
    if response.status_code == 200:
        return response.json()
    else:
        return None

def fetch_yearly_data(stock_no, year):
    all_data = []

    for month in range(1, 13):  # 從1月到12月
        data = fetch_monthly_data(stock_no, year, month)
        if data and data['stat'] == 'OK':
            # 將每個月的數據添加到列表中
            all_data.extend(data['data'])

    # 將數據轉換為DataFrame，並指定欄位名稱
    df = pd.DataFrame(all_data, columns=["日期", "成交股數", "成交金額", "開盤價", "最高價", "最低價", "收盤價", "漲跌價差", "成交筆數"])

    # 全部換成float以利後續計算
    # 使用 applymap 將 replace 和 astype 應用於每個單元格
    for col in ["成交股數", "成交金額", "開盤價", "最高價", "最低價", "收盤價", "成交筆數"]:
      try:
        df[col] = df[col].astype(str).str.replace(",", "").astype(float)
      except:
        pass

    # 將日期從民國年轉換為西元年格式
    df['日期'] = df['日期'].apply(convert_date)

    # 將日期欄位轉換為datetime格式，並重新排序數據
    df['日期'] = pd.to_datetime(df['日期'], format='%Y/%m/%d')
    df = df.sort_values(by='日期').reset_index(drop=True)

    return df

# 上市公司的公司代碼，拿取近一年(2024)的所有交易資料
for i in stock_dic:
  stock_no = i
  year = 2024
  print('現在進行到：',i)
  all_years_data = []
  df = fetch_yearly_data(stock_no, year)
  time.sleep(1) # 防止被認為是攻擊
  stock_dic[i]["歷史交易資料"] = df # 儲存個股交易資料


現在進行到： 2302
現在進行到： 2303
現在進行到： 2329
現在進行到： 2330
現在進行到： 2337
現在進行到： 2338
現在進行到： 2340
現在進行到： 2342
現在進行到： 2344
現在進行到： 2351
現在進行到： 2363
現在進行到： 2369
現在進行到： 2379
現在進行到： 2388
現在進行到： 2401
現在進行到： 2408
現在進行到： 2434
現在進行到： 2436
現在進行到： 2441
現在進行到： 2449
現在進行到： 2451
現在進行到： 2454
現在進行到： 2458
現在進行到： 2481
現在進行到： 3006
現在進行到： 3014
現在進行到： 3016
現在進行到： 3034
現在進行到： 3035
現在進行到： 3041
現在進行到： 3094
現在進行到： 3189
現在進行到： 3257
現在進行到： 3413
現在進行到： 3443
現在進行到： 3450
現在進行到： 3530
現在進行到： 3532
現在進行到： 3545
現在進行到： 3583
現在進行到： 3588
現在進行到： 3592
現在進行到： 3661
現在進行到： 3686
現在進行到： 3711
現在進行到： 4919
現在進行到： 4952
現在進行到： 4961
現在進行到： 4967
現在進行到： 4968
現在進行到： 5222
現在進行到： 5269
現在進行到： 5285
現在進行到： 5471
現在進行到： 6202
現在進行到： 6239
現在進行到： 6243
現在進行到： 6257
現在進行到： 6271
現在進行到： 6415
現在進行到： 6451
現在進行到： 6515
現在進行到： 6525
現在進行到： 6526
現在進行到： 6531
現在進行到： 6533
現在進行到： 6552
現在進行到： 6573
現在進行到： 6695
現在進行到： 6719
現在進行到： 6756
現在進行到： 6770
現在進行到： 6789
現在進行到： 6799
現在進行到： 6937
現在進行到： 6962
現在進行到： 8016
現在進行到： 8028
現在進行到： 8081
現在進行到： 8110
現在進行到： 8131
現在進行到： 8150
現在進行到： 8261
現在進行

In [ ]:
# 把stock_dic先存起來
import pickle

import pandas as pd

with open("drive/MyDrive/金融大數據分析/stock_dic.pkl", "wb") as file:
    pickle.dump(stock_dic, file)


In [ ]:
# 讀入stock_dic
import pickle

with open("drive/MyDrive/金融大數據分析/stock_dic.pkl", "rb") as file:
    stock_dic = pickle.load(file)
print(stock_dic["2302"])

{'類別': '上市', '歷史交易資料':             日期       成交股數        成交金額    開盤價    最高價    最低價    收盤價   漲跌價差  \
0   2024-01-02   109204.0   2010256.0  18.60  18.60  18.35  18.40  -0.05   
1   2024-01-03   178457.0   3263353.0  18.40  18.45  18.20  18.35  -0.05   
2   2024-01-04   349663.0   6323774.0  18.30  18.35  17.90  18.00  -0.35   
3   2024-01-05   265768.0   4766222.0  18.15  18.20  17.75  17.95  -0.05   
4   2024-01-08   301628.0   5428123.0  18.15  18.35  17.80  17.85  -0.10   
..         ...        ...         ...    ...    ...    ...    ...    ...   
237 2024-12-25   937310.0  17499515.0  18.35  19.00  18.20  18.80  +0.70   
238 2024-12-26  3293777.0  65220910.0  19.00  20.30  18.80  19.90  +1.10   
239 2024-12-27  1152030.0  22420947.0  19.90  20.05  19.00  19.35  -0.55   
240 2024-12-30   522558.0   9934132.0  19.35  19.35  18.75  18.75  -0.60   
241 2024-12-31   283959.0   5270163.0  18.85  18.85  18.30  18.70  -0.05   

       成交筆數  
0      52.0  
1     120.0  
2     199.0  
3     19

In [ ]:
# 1_2.使用鉅亨網進行個股的新聞爬蟲(只爬取近期最新的新聞)
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
from datetime import datetime, timedelta
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

def get_stocknews_url(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    news_ls = []
    url_ls = []

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    # 當前日期
    current_date = datetime.now()
    # 一週之前的日期
    one_week_ago = current_date - timedelta(days=7)

    # 模擬下拉到目標區域
    time.sleep(2)  # 等待頁面載入
    news_section = driver.find_element(By.CLASS_NAME, 'jsx-3426971566.tabs-container')  # 定位目標區域
    ActionChains(driver).move_to_element(news_section).perform()  # 移動到該元素
    time.sleep(2)  # 確保動態內容已加載

    # 抓取目標 `<a>` 標籤
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('a', class_='jsx-2831776980 container border')
    for article in articles:
        # 取得連結（href）
        link = article.get('href')
        # 取得標題文字
        title = article.find('h3').text.strip()
        # 取得時間與類別
        time_container = article.find('div', class_='time-container')
        news_time = time_container.find('span', class_='jsx-2831776980').text.strip()

        news_date = datetime.strptime(news_time, "%m/%d")  # 假設日期格式為 MM/DD
        # 因為只有月日，需補上當年的年份以便計算
        news_date = news_date.replace(year=current_date.year)

        # 篩選是否在最近 7 天內
        if one_week_ago <= news_date <= current_date:
            news_ls.append([title, news_date])  # 加入符合條件的新聞
            url_ls.append(link)
    return news_ls, url_ls

def get_newscontent(url_ls):
    content_ls = []
    if url_ls == []:
      return content_ls
    for url in url_ls:
      chrome_options = Options()
      chrome_options.add_argument('--headless')
      chrome_options.add_argument('--no-sandbox')
      chrome_options.add_argument('--disable-dev-shm-usage')
      driver = webdriver.Chrome(options=chrome_options)
      driver.get(url)

      # 模擬下拉到目標區域
      time.sleep(2)  # 等待頁面載入
      news_section = driver.find_element(By.CSS_SELECTOR, "main#article-container.c1tt5pk2")  # 定位目標區域
      ActionChains(driver).move_to_element(news_section).perform()  # 移動到該元素
      time.sleep(2)  # 確保動態內容已加載

      soup = BeautifulSoup(driver.page_source, 'html.parser')
      find_main = soup.find('main', {'id': 'article-container'})
      find_sec = find_main.find_all('p')
      content = ''
      for i in find_sec:
        content += i.text
      content_ls.append(content)
    return content_ls

now_ls = []
complete = ['2302', '2303', '2329', '2330', '2337', '2338']
for count, j in enumerate(stock_dic):
  if j in complete:
    continue
  # 設定鉅亨網的URL
  print('現在進行到:',j)
  now_ls.append(j)
  stock_code = j
  news_url = f'https://www.cnyes.com/twstock/{stock_code}/news/stock'

  news_ls, url_ls = get_stocknews_url(news_url)
  contnet_ls = get_newscontent(url_ls)
  for i in range(len(news_ls)):
    news_ls[i].append(contnet_ls[i])
  stock_dic[j]["個股新聞"] = news_ls

  if count % 5 == 0:
    # 把stock_dic先存起來
    with open("drive/MyDrive/金融大數據分析/stock_dic_withnews.pkl", "wb") as file:
        pickle.dump(stock_dic, file)
    print('已儲存:', *now_ls)
    now_ls = []
# 把剩餘的stock_dic存起來
with open("drive/MyDrive/金融大數據分析/stock_dic_withnews.pkl", "wb") as file:
    pickle.dump(stock_dic, file)
print('已儲存:', *now_ls)
print("全部新聞已爬取完畢")

現在進行到: 2340
現在進行到: 2342
現在進行到: 2344
現在進行到: 2351
現在進行到: 2363
已儲存: 2340 2342 2344 2351 2363
現在進行到: 2369
現在進行到: 2379
現在進行到: 2388
現在進行到: 2401
現在進行到: 2408
已儲存: 2369 2379 2388 2401 2408
現在進行到: 2434
現在進行到: 2436
現在進行到: 2441
現在進行到: 2449
現在進行到: 2451
已儲存: 2434 2436 2441 2449 2451
現在進行到: 2454
現在進行到: 2458
現在進行到: 2481
現在進行到: 3006
現在進行到: 3014
已儲存: 2454 2458 2481 3006 3014
現在進行到: 3016
現在進行到: 3034
現在進行到: 3035
現在進行到: 3041
現在進行到: 3094
已儲存: 3016 3034 3035 3041 3094
現在進行到: 3189
現在進行到: 3257
現在進行到: 3413
現在進行到: 3443
現在進行到: 3450
已儲存: 3189 3257 3413 3443 3450
現在進行到: 3530
現在進行到: 3532
現在進行到: 3545
現在進行到: 3583
現在進行到: 3588
已儲存: 3530 3532 3545 3583 3588
現在進行到: 3592
現在進行到: 3661
現在進行到: 3686
現在進行到: 3711
現在進行到: 4919
已儲存: 3592 3661 3686 3711 4919
現在進行到: 4952
現在進行到: 4961
現在進行到: 4967
現在進行到: 4968
現在進行到: 5222
已儲存: 4952 4961 4967 4968 5222
現在進行到: 5269
現在進行到: 5285
現在進行到: 5471
現在進行到: 6202
現在進行到: 6239
已儲存: 5269 5285 5471 6202 6239
現在進行到: 6243
現在進行到: 6257
現在進行到: 6271
現在進行到: 6415
現在進行到: 6451
已儲存: 6243 6257 6271 6415 6451
現在進行到: 651

### 2. 特徵生成
##### 使用前面得到的初級特徵進行其他特徵的生成

In [ ]:
# 在計算特徵前，先把所有值得資料型態轉為float
import pandas as pd

for i in stock_dic:
  cols = ["日期", "成交股數", "成交金額", "開盤價", "最高價", "最低價", "收盤價", "漲跌價差", "成交筆數"]
  for col in cols:
    # 找出所有唯一的資料型態
    unique_types = stock_dic[i]['歷史交易資料'][col].apply(type).unique()
    if unique_types == str:
      stock_dic[i]['歷史交易資料'][col] = pd.to_numeric(stock_dic[i]['歷史交易資料'][col], errors='coerce')

In [ ]:
# 2_1.生成個股基礎風險指數，以年化波動率和夏普比率作為風險指數
# 2_2.計算回報率(日收益率)
import numpy as np

for i in stock_dic:
  # 計算日收益率 (使用收盤價)
  stock_dic[i]['日收益率'] = stock_dic[i]['歷史交易資料']['收盤價'].pct_change()

  # 計算年化波動率 (日收益率的標準差乘以根號242，因一年大約242個交易日)
  daily_volatility = stock_dic[i]['日收益率'].std()
  annualized_volatility = daily_volatility * np.sqrt(242)

  # 基礎風險指數 (年化波動率)
  risk_index = annualized_volatility

  stock_dic[i]['基礎風險指數(年化波動率)'] = risk_index

  # 基礎風險指數(夏普比率)
  risk_free_rate = 0.01  # 假設無風險利率
  excess_returns = stock_dic[i]['日收益率'] - risk_free_rate / 242  # 每日超額回報
  sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(242)
  stock_dic[i]['基礎風險指數(夏普比率)'] = sharpe_ratio

<ipython-input-17-412d048143e6>:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_dic[i]['日收益率'] = stock_dic[i]['歷史交易資料']['收盤價'].pct_change()


In [ ]:
# 把stock_dic先存起來
import pickle

with open("drive/MyDrive/金融大數據分析/stock_dic.pkl", "wb") as file:
    pickle.dump(stock_dic, file)

In [ ]:
# 2_3.呼叫chatgpt api生成風險指數，並將temperature設為0，確保其每次結果不會相差太多
import openai

# 設定你的 API 密鑰
openai.api_key = ''
def call_openai_api(prompt):
    # 呼叫 OpenAI API
    completion = openai.ChatCompletion.create(
        model="chatgpt-4o-latest",
        messages=[
            {"role": "system", "content": prompt},
        ],
        temperature=0 #使其每次結果不會差太多
    )
prompt = '''
你是一位金融的風險控管專家，擅長多方面的考量各種因素。
現在我想請你做的事情是，考慮到現有的基礎風險比率(年化波動率和夏普比率)，通常只有數字上單方面的計算，而沒有考量到現實世界發生的新聞事件。
因此，我想請你幫每個股票，以夏普比率和年化波動率為基準，並結合所抓取的新聞(這些新聞皆為三日內發生的新聞)，
重新給出一個適當的風險指數，請回傳一個json檔，其中只需要包含各檔股票的代號和其新的風險指數即可。
'''
result = call_openai_api(prompt)
for i in result:
  # 將最終的風險指數存起來
  stock_dic[i]['最終風險指數'] = result[i]

### 3.投資組合選擇

In [ ]:
# 讀入stock_dic
import pickle

with open("drive/MyDrive/金融大數據分析/stock_dic.pkl", "rb") as file:
    stock_dic = pickle.load(file)


In [ ]:
# 使用基因演算法
import numpy as np
import pandas as pd
from scipy.special import softmax

# 假設我們有一個股票數據集
# 這裡的數據應該包含股票的歷史價格或回報率
data = stock_dic  # 載入股票數據

# 基因演算法參數
population_size = 5 # 每一代的染色體數量
num_stocks = len(data)  # 股票數量
num_generations = 100  # 總世代數
mutation_rate = 0.01   # 突變率
stock_code = list(data.keys()) # 股票代碼

# 初始化隨機染色體（每個基因代表是否選擇該股票）
def initialize_population(size, num_stocks):
    return np.random.randint(2, size=(size, num_stocks))

# 計算適應度函數（此例中使用年化回報率）
def fitness_function(chromosome, x=1, risk_mode='最終風險指數'):
    score = 0
    for count, i in enumerate(chromosome):
      if i == 1:
        code = stock_code[count]
        stock_score = data[code]['日收益率'].mean()
        weight = data[code]['歷史交易資料']['成交股數'].mean()
        risk = data[code][risk_mode]
        score += (stock_score - x*risk) * weight
    return score

# 選擇操作（輪盤賭選擇）
def selection(population, x=1, risk_mode='最終風險指數'):
    fitness_scores = np.array([fitness_function(chromosome, x, risk_mode) for chromosome in population])
    probabilities = fitness_scores / fitness_scores.sum()
    probabilities = softmax(probabilities)
    selected_indices = np.random.choice(range(population_size), size=population_size, p=probabilities)
    return population[selected_indices]

# 交叉操作
def crossover(parent1, parent2):
    point = np.random.randint(1, num_stocks - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

# 突變操作
def mutate(chromosome):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # 切換基因值
    return chromosome

# 主程序
def genes(epochs, x, risk_mode, num_generations):
  choice = []
  for epoch in range(epochs):
    population = initialize_population(population_size, num_stocks)

    for generation in range(num_generations):
        selected_population = selection(population, x)

        next_generation = []

        # 確保選擇偶數個父母以進行交叉
        for i in range(0, population_size - 1, 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1]
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1))
            next_generation.append(mutate(child2))

        # 如果 population_size 是奇數，則將最後一個染色體直接添加到下一代
        if population_size % 2 == 1:
            next_generation.append(selected_population[-1])

        population = np.array(next_generation)

    # 最佳解
    best_chromosome = max(population, key=fitness_function)
    choice.append(best_chromosome)
  return choice

### 實驗與結果分析

In [ ]:
# 先寫一個func評估各投資組合的好壞
stock_code = list(stock_dic.keys())
taiwan_returns = 0.287 #台股2024的市場回報率
def evaluate(choice):
  # 計算詹森指標
  # 計算投資組合和市場的平均回報率
  returns = []
  counts = 0
  risk_free_rate = 0.02
  for count, i in enumerate(choice):
    if i == 1:
      code = stock_code[count]
      returns.append(stock_dic[code]['日收益率'].mean())
      counts += 1
  market_returns = [taiwan_returns for _ in range(counts)]
  Rp = np.mean(returns)
  Rm = taiwan_returns

  # 計算貝他係數
  '''
  covariance = np.cov(returns, market_returns)[0][1]
  beta = covariance / np.var(market_returns)
  '''

  # 計算詹森指標
  alpha = (Rp - risk_free_rate) - -(Rm - risk_free_rate)
  return alpha

In [ ]:
np.var([1,1,1,1,1])

0.0

### 實驗一
##### 使用不同的風險指數做基因演算

In [ ]:
epochs = 10
x = 1
risk_mode = ['最終風險指數','基礎風險指數(夏普比率)','基礎風險指數(年化波動率)']
num_generations = 500

for i in risk_mode:
  choices = genes(epochs, x, i, num_generations) # 長度與epochs相同
  s_score = 0
  for j in choices:
    s_score += evaluate(j)
  print(i,"average score:", s_score/len(choices))


最終風險指數 average score: 0.12696186641267795
基礎風險指數(夏普比率) average score: 0.08687608278570727
基礎風險指數(年化波動率) average score: 0.0869749857706328



### 實驗二
##### 使用不同的gpt模板

In [ ]:
# 最終模板
prompt = '''
你是一位金融的風險控管專家，擅長多方面的考量各種因素。現在我想請你做的事情是，考慮到現有的基礎風險比率(年化波動率和夏普比率)，通常只有數字上單方面的計算，而沒有考量到現實世界發生的新聞事件。
因此，我想請你幫每個股票，以夏普比率和年化波動率為基準，並結合所抓取的新聞(這些新聞皆為三日內發生的新聞)，重新給出一個適當的風險指數，並且請遵循以下指示：
1. 請先對各篇新聞最深度的語意分析，以判斷該新聞對於風險的影響，請勿直接以新聞的篇數做為風險影響的大小依據
2. 請回傳ㄧ個json檔，其中只需要包含各檔股票的代號和其新的風險指數即可。
'''
result = call_openai_api(prompt)
new_stock_dic = stock_dic
for i in result:
  # 將最終的風險指數存起來
  new_stock_dic[i]['最終風險指數'] = result[i]

In [ ]:
data = new_stock_dic #用新的最終風險分數
epochs = 10
x = 1
risk_mode = '最終風險指數'
num_generations = 500

choices = genes(epochs, x, risk_mode, num_generations) # 長度與epochs相同
s_score = 0
for j in choices:
  s_score += evaluate(j)
print(risk_mode,"average score:", s_score/len(choices))


最終風險指數 average score: 0.14278456631811562
